**Pybullet Quickstart Guide** \
https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit?tab=t.0#heading=h.3ei4cod3v112 \
**Pybullet example** \
https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet/examples

In [ ]:
import numpy as np
import pybullet as p
import pybullet_data

# Open PyBullet GUI

### * Connect to the PyBullet GUI

In [ ]:
ClientId = p.connect(p.GUI)

### * Set GUI configurations

In [ ]:
p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)                        # Enable/Disable a default GUI
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)                    # Enable/Disable a light effect
p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, 0)         # Show/Hide a RGB buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, 0)       # Show/Hide a Depth buffer preview window
p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, 0)  # Show/Hide a Seg-map buffer preview window

### * Add plane & plot world coordinate frame

In [ ]:
PlaneId = p.loadURDF("plane.urdf")

In [ ]:
WorldXDebugLineID = p.addUserDebugLine(lineFromXYZ=[0, 0, 0], lineToXYZ=[0.3, 0, 0], lineColorRGB=[1, 0, 0],
                                       lineWidth=5, physicsClientId=ClientId)
WorldYDebugLineID = p.addUserDebugLine(lineFromXYZ=[0, 0, 0], lineToXYZ=[0, 0.3, 0], lineColorRGB=[0, 1, 0],
                                       lineWidth=5, physicsClientId=ClientId)
WorldZDebugLineID = p.addUserDebugLine(lineFromXYZ=[0, 0, 0], lineToXYZ=[0, 0, 0.3], lineColorRGB=[0, 0, 1],
                                       lineWidth=5, physicsClientId=ClientId)

### * Set camera

In [ ]:
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=45, cameraPitch=-30, cameraTargetPosition=[0, 0, 0.5])

### * Set physics scene

In [ ]:
p.setGravity(0, 0, -9.81) # set a gravity vector
p.setTimeStep(1./240)     # set a simulation time step (240 Hz)

In [ ]:
p.setRealTimeSimulation(1) # start real-time physics simulation

### Keyboard shortcuts

**ctrl $+$ LB**: Rotate view \
**ctrl $+$ RB**: Translate view \
**ctrl $+$ MB**: Zoom view in/out \
**MB $\pm$ wheel**: Zoom view in/out \

**w**: Toggle wireframe \
**j**: Show links and joints frames as RGB lines \
**k**: Show joint axes as a black line \

# Add/Remove object in PyBullet

### * Add box shape object (10cm * 10cm * 10cm)

##### - create visual/collision shape

In [ ]:
obj_color = [1, 0, 0, 1]
obj_dim = [0.10/2, 0.10/2, 0.10/2]

visualShapeId = p.createVisualShape(shapeType=p.GEOM_BOX, 
                                    halfExtents=obj_dim, 
                                    rgbaColor=obj_color)

collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_BOX, 
                                          halfExtents=obj_dim)

print("visualShapeId: ", visualShapeId)
print("collisionShapeId: ", collisionShapeId)

##### -create rigid body

In [ ]:
boxShapeId = p.createMultiBody(baseMass=1,
                               baseInertialFramePosition =[0, 0, 0],
                               baseVisualShapeIndex=visualShapeId,
                               baseCollisionShapeIndex=collisionShapeId,
                               basePosition=[0, 0, 1])

print("boxShapeId: ", boxShapeId)

##### - remove rigid body

In [ ]:
p.removeBody(boxShapeId)

### * Add mesh object

In [ ]:
obj_color = [1, 1, 1, 1]
obj_shift = [0, 0.02, 0]
obj_scale = [0.4, 0.4, 0.4]

visualShapeId = p.createVisualShape(shapeType=p.GEOM_MESH,
                                    fileName="duck.obj",
                                    rgbaColor=obj_color,
                                    specularColor=[1, 1, 1],
                                    visualFramePosition=obj_shift,
                                    meshScale=obj_scale)

collisionShapeId = p.createCollisionShape(shapeType=p.GEOM_MESH,
                                          fileName="duck_vhacd.obj",
                                          collisionFramePosition=obj_shift,
                                          meshScale=obj_scale)

print("visualShapeId: ", visualShapeId)
print("collisionShapeId: ", collisionShapeId)

In [ ]:
meshShapeId = p.createMultiBody(baseMass=1,
                               baseInertialFramePosition =[0, 0, 0],
                               baseVisualShapeIndex=visualShapeId,
                               baseCollisionShapeIndex=collisionShapeId,
                               basePosition=[0, 0, 1])

print("meshShapeId: ", meshShapeId)

In [ ]:
p.removeBody(meshShapeId)

# Import robot from URDF

In [ ]:
urdf_path = "./indy7_v2/model.urdf"
base_pos = [0, 0, 0]
base_quat = p.getQuaternionFromEuler([0, 0, 0])

flags = p.URDF_USE_INERTIA_FROM_FILE + p.URDF_USE_SELF_COLLISION + p.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT

robotId = p.loadURDF(urdf_path, 
                     basePosition=base_pos, 
                     baseOrientation=base_quat,
                     flags=flags, 
                     physicsClientId=ClientId)

### get robot's joint info

In [ ]:
numJoints = p.getNumJoints(robotId, physicsClientId=ClientId)
print("numJoints: ", numJoints)

def getPybulletJointType(JointType):
    if JointType == p.JOINT_FIXED:
        return 'fixed'
    elif JointType == p.JOINT_REVOLUTE:
        return 'revolute'
    elif JointType == p.JOINT_PRISMATIC:
        return 'prismatic'
    else:
        return 'etc'

MovableJointIdx = []
EEJointIdx = None

for jointIndex in range(numJoints):
    JointInfo = p.getJointInfo(robotId, jointIndex=jointIndex, physicsClientId=ClientId)
    print("\n*** Joint {} ***".format(jointIndex))
    print("JointName: ", JointInfo[1])
    print("JointType: ", getPybulletJointType(JointInfo[2]))
    
    if getPybulletJointType(JointInfo[2]) in ['revolute', 'prismatic']:
        MovableJointIdx.append(jointIndex)
    elif getPybulletJointType(JointInfo[2]) == 'fixed':
        EEJointIdx = jointIndex

print("\nnumMovableJoints: ", len(MovableJointIdx))

### get robot's joint states

In [ ]:
JointStates = p.getJointStates(robotId, jointIndices=MovableJointIdx, physicsClientId=ClientId)

for JointIdx, JointState in zip(MovableJointIdx, JointStates):
    print("\n*** Joint {} ***".format(JointIdx))
    print("JointPosition (rad): ", JointState[0])
    print("JointVelocity (rad/s): ", JointState[1])

### get robot's link states

In [ ]:
LinkStates = p.getLinkStates(robotId, linkIndices=MovableJointIdx+[EEJointIdx], physicsClientId=ClientId)

for LinkIdx, LinkState in zip(MovableJointIdx+[EEJointIdx], LinkStates):
    print("\n*** Link {} ***".format(LinkIdx))
    print("LinkPosition (m): ", LinkState[4])
    print("LinkOrientation (deg): ", p.getEulerFromQuaternion(LinkState[5]))

# Control robot

### Position Control (Simple PD control)

In [ ]:
p.setJointMotorControlArray(robotId, 
                            jointIndices=MovableJointIdx, 
                            controlMode=p.POSITION_CONTROL, 
                            physicsClientId=ClientId)

##### - change PD gain

In [ ]:
positionGains = [0.01]*len(MovableJointIdx)
velocityGains = [0.01]*len(MovableJointIdx)

p.setJointMotorControlArray(robotId, 
                            jointIndices=MovableJointIdx, 
                            controlMode=p.POSITION_CONTROL, 
                            positionGains=positionGains,
                            velocityGains=velocityGains,
                            physicsClientId=ClientId)

##### -change target joint positions

In [ ]:
targetPositions = np.random.randn(len(MovableJointIdx))

p.setJointMotorControlArray(robotId, 
                            jointIndices=MovableJointIdx, 
                            controlMode=p.POSITION_CONTROL, 
                            targetPositions=targetPositions,
                            physicsClientId=ClientId)